In [1]:
import os
import sqlite3
import subprocess
import json

# SQLite-Datenbank erstellen oder öffnen
def init_db(db_name="media_metadata.db"):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS media_metadata (
            id TEXT PRIMARY KEY,
            url TEXT,
            title TEXT,
            description TEXT,
            duration INTEGER,
            view_count INTEGER,
            like_count INTEGER,
            repost_count INTEGER,
            comment_count INTEGER,
            uploader TEXT,
            uploader_id TEXT,
            channel TEXT,
            channel_id TEXT,
            channel_url TEXT,
            track TEXT,
            album TEXT,
            artists TEXT,
            timestamp INTEGER,
            extractor TEXT
        )
    ''')
    conn.commit()
    return conn

# Metadaten extrahieren mit yt-dlp
def extract_metadata(url):
    try:
        result = subprocess.run(
            ["yt-dlp", "--flat-playlist", "--dump-single-json", url],
            capture_output=True,
            text=True
        )
        if result.returncode == 0:
            return json.loads(result.stdout)
        else:
            print(f"Fehler beim Abrufen von {url}: {result.stderr}")
            return None
    except Exception as e:
        print(f"Fehler: {e}")
        return None

# Einzelne Video-Metadaten in SQLite speichern
def save_video_metadata(conn, video):
    cursor = conn.cursor()
    try:
        cursor.execute('''
            INSERT OR REPLACE INTO media_metadata (
                id, url, title, description, duration, view_count,
                like_count, repost_count, comment_count, uploader,
                uploader_id, channel, channel_id, channel_url, track,
                album, artists, timestamp, extractor
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            video.get("id"),
            video.get("url"),
            video.get("title"),
            video.get("description"),
            video.get("duration"),
            video.get("view_count"),
            video.get("like_count"),
            video.get("repost_count"),
            video.get("comment_count"),
            video.get("uploader"),
            video.get("uploader_id"),
            video.get("channel"),
            video.get("channel_id"),
            video.get("channel_url"),
            video.get("track"),
            video.get("album"),
            ", ".join(video.get("artists", [])),
            video.get("timestamp"),
            video.get("extractor")
        ))
        conn.commit()
    except sqlite3.Error as e:
        print(f"SQLite-Fehler: {e}")

# Playlist verarbeiten und Metadaten speichern
def process_playlist_metadata(conn, playlist_metadata):
    if not playlist_metadata or "entries" not in playlist_metadata:
        print("Keine Videos in der Playlist gefunden.")
        return

    for video in playlist_metadata["entries"]:
        save_video_metadata(conn, video)

# Links aus Datei lesen und verarbeiten
def process_links(file_name="links.txt", db_name="media_metadata.db"):
    conn = init_db(db_name)
    if not os.path.exists(file_name):
        print(f"Datei {file_name} nicht gefunden.")
        return

    with open(file_name, "r") as file:
        links = [line.strip() for line in file if line.strip()]

    for url in links:
        print(f"Verarbeite URL: {url}")
        playlist_metadata = extract_metadata(url)
        if playlist_metadata:
            process_playlist_metadata(conn, playlist_metadata)

    conn.close()

if __name__ == "__main__":
    process_links()


Verarbeite URL: https://www.tiktok.com/@afd_bestofclips


In [10]:
# Datenbankinhalt in einen Pandas-DataFrame laden
import pandas as pd

def db_to_dataframe(db_name="media_metadata.db"):
    conn = sqlite3.connect(db_name)
    query = "SELECT * FROM media_metadata"
    try:
        df = pd.read_sql_query(query, conn)
        return df
    except sqlite3.Error as e:
        print(f"Fehler: {e}")
        return None
    finally:
        conn.close()

df = db_to_dataframe()

df

,id,url,title,description,duration,view_count,like_count,repost_count,comment_count,uploader,uploader_id,channel,channel_id,channel_url,track,album,artists,timestamp,extractor
0,7381136306660805921,https://www.tiktok.com/@afd_bestofclips/video/...,,,63.0,442500,13300,4970,5960,afd_bestofclips,7230041391786640389,AfD_Fanpage,MS4wLjABAAAApQTg0irYglPHy0uepOHmd14a6Yth5Rko_-...,https://www.tiktok.com/@MS4wLjABAAAApQTg0irYgl...,Originalton,None,AfD_Fanpage,1718554724,None
1,7380108424270318880,https://www.tiktok.com/@afd_bestofclips/video/...,Lanz konfrontiert Saskia Esken (SPD) mit einer...,Lanz konfrontiert Saskia Esken (SPD) mit einer...,87.0,138400,6666,818,1861,afd_bestofclips,7230041391786640389,AfD_Fanpage,MS4wLjABAAAApQTg0irYglPHy0uepOHmd14a6Yth5Rko_-...,https://www.tiktok.com/@MS4wLjABAAAApQTg0irYgl...,Originalton,None,AfD_Fanpage,1718315402,None
2,7380045789420195105,https://www.tiktok.com/@afd_bestofclips/video/...,Der Anfang vom Video ist am besten 😂,Der Anfang vom Video ist am besten 😂,45.0,89400,8750,135,467,afd_bestofclips,7230041391786640389,AfD_Fanpage,MS4wLjABAAAApQTg0irYglPHy0uepOHmd14a6Yth5Rko_-...,https://www.tiktok.com/@MS4wLjABAAAApQTg0irYgl...,Originalton,None,AfD_Fanpage,1718300817,None
3,7378537749986413857,https://www.tiktok.com/@afd_bestofclips/video/...,Grüne -8%! #AfD jetzt zweitstärkste Partei! Da...,Grüne -8%! #AfD jetzt zweitstärkste Partei! Da...,39.0,715000,67600,6564,6589,afd_bestofclips,7230041391786640389,AfD_Fanpage,MS4wLjABAAAApQTg0irYglPHy0uepOHmd14a6Yth5Rko_-...,https://www.tiktok.com/@MS4wLjABAAAApQTg0irYgl...,Originalton,None,AfD_Fanpage,1717949701,None
4,7377559901456878881,https://www.tiktok.com/@afd_bestofclips/video/...,Das hat gesessen! Schaut mal am Ende: Warum zi...,Das hat gesessen! Schaut mal am Ende: Warum zi...,74.0,650600,42100,2915,2934,afd_bestofclips,7230041391786640389,AfD_Fanpage,MS4wLjABAAAApQTg0irYglPHy0uepOHmd14a6Yth5Rko_-...,https://www.tiktok.com/@MS4wLjABAAAApQTg0irYgl...,Originalton,None,AfD_Fanpage,1717722068,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,7243322513629924634,https://www.tiktok.com/@afd_bestofclips/video/...,#AfD#nurnochAfD#AliceWeidel,#AfD#nurnochAfD#AliceWeidel,110.0,2320,263,8,7,afd_bestofclips,7230041391786640389,AfD_Fanpage,MS4wLjABAAAApQTg0irYglPHy0uepOHmd14a6Yth5Rko_-...,https://www.tiktok.com/@MS4wLjABAAAApQTg0irYgl...,Originalton,None,AfD_Fanpage,1686467468,None
187,7243180795366477083,https://www.tiktok.com/@afd_bestofclips/video/...,#nurnochAfD#Afd,#nurnochAfD#Afd,49.0,68900,5626,209,684,afd_bestofclips,7230041391786640389,AfD_Fanpage,MS4wLjABAAAApQTg0irYglPHy0uepOHmd14a6Yth5Rko_-...,https://www.tiktok.com/@MS4wLjABAAAApQTg0irYgl...,Originalton,None,AfD_Fanpage,1686434459,None
188,7243152949545880859,https://www.tiktok.com/@afd_bestofclips/video/...,2025 richtig Wählen! #nurnochAfD#AfD#AliceWeidel,2025 richtig Wählen! #nurnochAfD#AfD#AliceWeidel,144.0,2600,203,15,7,afd_bestofclips,7230041391786640389,AfD_Fanpage,MS4wLjABAAAApQTg0irYglPHy0uepOHmd14a6Yth5Rko_-...,https://www.tiktok.com/@MS4wLjABAAAApQTg0irYgl...,Originalton,None,AfD_Fanpage,1686427975,None
189,7243146844656782618,https://www.tiktok.com/@afd_bestofclips/video/...,2025 richtig Wählen! #nurnochAfD#AfD#AliceWeidel,2025 richtig Wählen! #nurnochAfD#AfD#AliceWeidel,82.0,1674,173,2,6,afd_bestofclips,7230041391786640389,AfD_Fanpage,MS4wLjABAAAApQTg0irYglPHy0uepOHmd14a6Yth5Rko_-...,https://www.tiktok.com/@MS4wLjABAAAApQTg0irYgl...,Originalton,None,AfD_Fanpage,1686426554,None
